In [1]:
import pandas as pd

robot_data = pd.read_csv('robot_dataset.csv')
robot_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             493 non-null    int64  
 1   adult                  493 non-null    bool   
 2   backdrop_path          344 non-null    object 
 3   belongs_to_collection  98 non-null     object 
 4   budget                 493 non-null    int64  
 5   genres                 493 non-null    object 
 6   homepage               148 non-null    object 
 7   id                     493 non-null    int64  
 8   imdb_id                453 non-null    object 
 9   original_language      493 non-null    object 
 10  original_title         493 non-null    object 
 11  overview               488 non-null    object 
 12  popularity             493 non-null    float64
 13  poster_path            447 non-null    object 
 14  production_companies   493 non-null    object 
 15  produc

In [2]:
text_data = robot_data[['title', 'tagline', 'overview']].dropna()
text_data.head()

,title,tagline,overview
0,Finch,Take an unforgettable journey.,"On a post-apocalyptic Earth, a robot, built to..."
1,Godzilla vs. Kong,One Will Fall,"In a time when monsters walk the Earth, humani..."
2,Pacific Rim,"To Fight Monsters, We Created Monsters",A ragtag band of humans band together in the y...
3,Real Steel,"If you get one shot, make it real.",Charlie Kenton is a washed-up fighter who reti...
4,Outside the Wire,Defiant by design.,"In the near future, a drone pilot is sent into..."


In [3]:
import string
import re

remove_string = '\w*robot\w*'
#remove original keywords
text_data['overview_processed'] = text_data['overview'].map(lambda x: re.sub(r'{}'.format(remove_string), '', x))
# Remove punctuation
text_data['overview_processed'] = text_data['overview_processed'].map(lambda x: re.sub(r'[,\.!?]', '', x))
# Convert the titles to lowercase
text_data['overview_processed'] = text_data['overview_processed'].map(lambda x: x.lower())
# # Print out the first rows of papers
text_data['overview_processed'].head()

0    on a post-apocalyptic earth a  built to protec...
1    in a time when monsters walk the earth humanit...
2    a ragtag band of humans band together in the y...
3    charlie kenton is a washed-up fighter who reti...
4    in the near future a drone pilot is sent into ...
Name: overview_processed, dtype: object

In [4]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc))
             if word not in stop_words] for doc in texts]

data = text_data['overview_processed'].values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

['post', 'apocalyptic', 'earth', 'built', 'protect', 'life', 'dying', 'creator', 'beloved', 'dog', 'learns', 'life', 'love', 'friendship', 'means', 'human']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1)]


In [6]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.010*"world" + 0.007*"must" + 0.004*"city" + 0.004*"day" + 0.004*"future" '
  '+ 0.004*"cyborg" + 0.004*"war" + 0.004*"secret" + 0.004*"first" + '
  '0.003*"one"'),
 (1,
  '0.006*"must" + 0.005*"new" + 0.005*"kung" + 0.004*"named" + 0.004*"future" '
  '+ 0.004*"satan" + 0.004*"world" + 0.004*"human" + 0.004*"one" + '
  '0.004*"chitti"'),
 (2,
  '0.008*"earth" + 0.008*"space" + 0.007*"must" + 0.004*"godzilla" + '
  '0.004*"one" + 0.004*"life" + 0.004*"future" + 0.003*"unit" + 0.003*"new" + '
  '0.003*"powerful"'),
 (3,
  '0.008*"life" + 0.008*"one" + 0.008*"earth" + 0.007*"find" + 0.005*"quaid" + '
  '0.004*"human" + 0.004*"police" + 0.004*"world" + 0.004*"planet" + '
  '0.004*"future"'),
 (4,
  '0.006*"save" + 0.005*"earth" + 0.004*"stop" + 0.004*"series" + '
  '0.004*"world" + 0.004*"discovers" + 0.004*"planet" + 0.004*"must" + '
  '0.004*"becomes" + 0.004*"fight"'),
 (5,
  '0.007*"earth" + 0.006*"human" + 0.005*"time" + 0.005*"one" + 0.005*"big" + '
  '0.004*"last" + 0.004*"

In [7]:
import pyLDAvis.gensim_models
import pickle
import pyLDAvis
import os
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('./ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, './ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

c:\users\marti\pycharmprojects\film-data-explorer\venv\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.020269 -0.100732       1        1  14.469770
3      0.117511  0.000847       2        1  11.036146
9     -0.068536  0.009508       3        1  10.442720
7      0.040835  0.047982       4        1  10.138697
5      0.021764  0.022131       5        1   9.614437
2     -0.037233  0.085789       6        1   9.480654
0     -0.055997 -0.015321       7        1   9.473707
8      0.026083 -0.008466       8        1   9.387856
4     -0.017502  0.024928       9        1   8.793922
1     -0.047194 -0.066666      10        1   7.162091, topic_info=         Term       Freq      Total Category  logprob  loglift
102      must  23.000000  23.000000  Default  30.0000  30.0000
119     space  17.000000  17.000000  Default  29.0000  29.0000
36      alien  12.000000  12.000000  Default  28.0000  28.0000
269      find  13.000000  13.000000  Default  27.0000  27.0000
28     planet  18.000000  18.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
551     plans   1.344430  12.420898  Topic10  -5.9229   0.4130
1277    robot   1.275751   8.034238  Topic10  -5.9753   0.7962
195   control   1.262667   8.965625  Topic10  -5.9856   0.6762
490       boy   1.238061  11.753126  Topic10  -6.0053   0.3858
378     super   1.225317  10.413176  Topic10  -6.0157   0.4965

[707 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
2883      4  0.739126    abandon
902       3  0.559151  abilities
1656     10  0.788779    ability
1317      1  0.378724      abyss
1317     10  0.378724      abyss
...     ...       ...        ...
301       6  0.081782      young
301       7  0.163564      young
301       8  0.081782      young
301       9  0.081782      young
301      10  0.081782      young

[1333 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 4, 10, 8, 6, 3, 1, 9, 5, 2])